In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

GPU/CPUのパフォーマンス比較

In [2]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)

def gpu():
  sess.run(net_gpu)

# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
8.707231875999923
GPU (s):
0.8626742569999806
GPU speedup over CPU: 10x


Kerasをインストールする

データセットの準備

In [3]:
!pip install -q keras
import keras

Using TensorFlow backend.


In [4]:
from keras.datasets import mnist
mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

Keras MNISTをCNNで推定するサンプルコードを実行する

In [5]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.2664 - acc: 0.9189 - val_loss: 0.0579 - val_acc: 0.9815
Epoch 2/12
60000/60000 [==============================] - 10s 171us/step - loss: 0.0934 - acc: 0.9722 - val_loss: 0.0446 - val_acc: 0.9853
Epoch 3/12
 5888/60000 [=>............................] - ETA: 8s - loss: 0.0712 - acc: 0.9798

60000/60000 [==============================] - 10s 170us/step - loss: 0.0699 - acc: 0.9795 - val_loss: 0.0325 - val_acc: 0.9890
Epoch 4/12
60000/60000 [==============================] - 10s 171us/step - loss: 0.0558 - acc: 0.9837 - val_loss: 0.0294 - val_acc: 0.9903
Epoch 5/12
56064/60000 [===========================>..] - ETA: 0s - loss: 0.0497 - acc: 0.9852

60000/60000 [==============================] - 10s 169us/step - loss: 0.0490 - acc: 0.9853 - val_loss: 0.0294 - val_acc: 0.9905
Epoch 6/12
60000/60000 [==============================] - 10s 169us/step - loss: 0.0439 - acc: 0.9874 - val_loss: 0.0270 - val_acc: 0.9918
Epoch 7/12
60000/60000 [==============================] - 10s 168us/step - loss: 0.0387 - acc: 0.9884 - val_loss: 0.0287 - val_acc: 0.9905
Epoch 8/12
 4736/60000 [=>............................] - ETA: 8s - loss: 0.0338 - acc: 0.9905

60000/60000 [==============================] - 10s 170us/step - loss: 0.0361 - acc: 0.9894 - val_loss: 0.0293 - val_acc: 0.9903
Epoch 9/12
60000/60000 [==============================] - 10s 170us/step - loss: 0.0341 - acc: 0.9895 - val_loss: 0.0273 - val_acc: 0.9908
Epoch 10/12
55424/60000 [==========================>...] - ETA: 0s - loss: 0.0308 - acc: 0.9899

60000/60000 [==============================] - 10s 171us/step - loss: 0.0314 - acc: 0.9899 - val_loss: 0.0258 - val_acc: 0.9918
Epoch 11/12
60000/60000 [==============================] - 10s 169us/step - loss: 0.0297 - acc: 0.9911 - val_loss: 0.0329 - val_acc: 0.9887
Epoch 12/12
60000/60000 [==============================] - 10s 169us/step - loss: 0.0268 - acc: 0.9915 - val_loss: 0.0274 - val_acc: 0.9915
Test loss: 0.027359745790594206
Test accuracy: 0.9915


FuseでGoogle Driveをマウント

In [6]:
# google-drive-ocamlfuseのインストール
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colab用のAuth token作成
from google.colab import auth
auth.authenticate_user()

# Drive FUSE library用のcredential生成
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# drive/ を作り、そこにGoogle Driveをマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


確認

In [7]:
!ls drive/

app-release-unsigned.apk  Untitled0.ipynb     見た映画履歴.ods
bot config		  京都旅行.desktop    視聴したアニメ履歴.ods
Colab Notebooks		  経費.ods	      読んだマンガ履歴.ods
dotstamp_DB定義書.ods	  英語1.odt
TTOL_CSV定義書.ods	  英語のレッスン.odt
